In [1]:
import pickle
import numpy as np
import sys,json,os
sys.path.append('..')
from project.recoil import Nuclear as Nr
from project.recoil import p50
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'sans'
matplotlib.rcParams['mathtext.fontset'] = 'dejavusans'

In [2]:
MWdict = pickle.load(open('../Output/MW_dict.pkl','rb'))
mwd = MWdict['vdf_RCfit']
MWlike_dict = pickle.load(open('../Output/MWlike_dict.pkl','rb'))
mwld = MWlike_dict['vdf_RCfit']
mwgals = list(mwld.keys())
nrd = pickle.load(open('../Output/Nuclear_dict.pkl','rb'))

# 2. Monte Carlo Method

In [3]:
def mk_mocks(key, nr_init='default', Nmocks=1000, seed=5100, 
             mdm_ori=4., sdm_ori=1e-45):
    """
    This function creates Nmocks number of mock samples as 
    returned by project.recoil.Nuclear().mocksamples().
    Attributes:
    ------------
    key: (str) keyword for the type of mocks to be made.
        possible keys are ['stat', 'MW', 'MWlike']
    nr_init: An object of the project.recoil.Nuclear() 
        namesapce. This is the initial setup from where
        the mocks will be created depening on the key.
    Nmocks: (int) number of mocks to be made.
    seed: (int), this will be used to create Nmocks number
        of random seeds for creating the mocks.
    mdm_ori, sdm_ori: Original values of m (in GeV) and 
        σp (in cm^-2) to create mock with.
    """
    np.random.seed(seed=seed)
    seeds = np.random.randint(low=1000, high=10000, size=Nmocks)

    if nr_init == 'default':
        nr_init = Nr('Xe', vE=mwd['vE'], 
                     vdfE=mwd['vdfE_50'],
                     vesc=p50(mwd['vescs']),
                     vcirc=p50(mwd['vcircs']),
                     rhosun=p50(mwd['rhosuns']),
                     Ethr=0.1, Eroi=5.0, ω=10.0)
    
    mocks, nrs = [], []
    for i in tqdm(range(Nmocks)):
        if key == 'stat':
            nr = nr_init
        elif (key == 'MW') or (key == 'MWlike'):
            if key == 'MW':
                gal_dict = mwd
            elif key == 'MWlike':
                gal_no = np.random.randin(0, len(mwgals)+1, size=1)
                gal_dict = mwd if gal_no == 0 else mwld[mwgals[gal_no]]
                
            chain_len = len(gal_dict['vdfEs'])
            chain_no = np.random.randint(0, chain_len, size=1)[0]
            nr = Nr(element=nr_init.element, 
                    vE=gal_dict['vE'],
                    vdfE=gal_dict['vdfEs'][chain_no],
                    vesc=gal_dict['vescs'][chain_no],
                    vcirc=gal_dict['vcircs'][chain_no],
                    rhosun=gal_dict['rhosuns'][chain_no],
                    Ethr=nr_init.Ethr, 
                    Eroi=nr_init.Eroi,
                    ω=nr_init.ω)
            
        mock = nr.mocksample(4., 1e-45, 0.001, Ntot='poisson',
                             seed=seeds[i])
        nrs.append(nr)
        mocks.append(mock)

    return (mocks, nrs)

In [88]:
from scipy.optimize import minimize, fsolve
import math

def bl̂_func(bl, *args):
    nobs, λsg, ΔE, exp = args
    return np.mean(nobs/(λsg + exp*bl*ΔE)) - 1.0

def lsdm̂_func(lsdm, *args):
    sdm = math.pow(10, lsdm)
    nobs, λsg0, ΔE, exp = args
    λsg = λsg0*sdm/1e-46
    args[1] = λsg
    blm = fsolve(bl̂_func, [0.001], args=args)[0]
    return nlL1(blm, nobs, λsg, exp)

def nlL1(bl, nobs, λsg, exp):
    λ = λsg + exp*ΔE*bl
    lL = np.sum(nobs*np.log(λ)) - np.sum(λ)
    return -lL

def get_MLEs(mocks, nr_inits):
    """
    Returns the MLEs of each mocks
    Attributes:
    ------------
    mocks: list of mocks with elements the result of
        project.recoil.Nuclear().mocksamples()
    nr_inits: list of the nr_init requried for initializing
        the class MLE. If len(nr_init) == len(mocks), then,
        MLE of mock[i] is found with nr_inits[i]. Else the 
        first element of nr_inits is fixed for all mock[i]
        in mocks. This list must not be empty.
    """
    Mdm, Sdm, Bl = [],[],[]
    for i,mock in enumerate(mocks):
        if len(nr_inits) == len(mocks):
            nr_init = nr_inits[i]
        else:
            nr_init = nr_inits[0]
            
        mle = MLE(mock, nr_init)
        Mdm.append(mle.mdm̂)
        Sdm.append(mle.sdm̂)
        Bl.append(mle.bl̂)
    return np.array([Mdm, Sdm, Bl]).T
    
class MLE:
    """
    Finds the Maximum Likelihood Estimators of m, σp, bl.
    """
    def __init__(self, mock, nr_init):
        """
        Initialization...
        Attributes
        ------------
        mock: details of mock sample as returned by 
            project.recoil.Nuclear().mocksample()
        nr_inti: Namesapce of project.recoil.Nuclear() class
            which fixes the experimental setup and the VDF 
            information (defaults to the best fit MW VDF).
        """
        self.mock = mock
        self.nr = nr_init
        self.Mdm = np.logspace(0,1,10)
        self.nobs = self.mock['binned_Esample']
        self.ΔE = (self.nr.Eroi - self.nr.Ethr)/self.mock['Nbins']
        self.exp = self.nr.exposure

        self.λsg0s = self.get_λsg0s()
        self.best = self.min_mdmsdmbl()
        self.mdm̂, self.sdm̂, self.bl̂ = self.best

    def get_λsg0s(self):
        λsg0s = []
        for mdm in Mdm:
            binT = self.nr.binTot(mdm, 1e-46, 0., mock['bin_edges'])
            λsg0s.append(binT['Neachbin'])
        return λsg0s

    def λsg0(self, mdm):
        indx = np.where(self.Mdm == mdm)
        return self.λsg0s[indx]

    def λsg(self, mdm, sdm):
        return self.λsg0(mdm)*sdm/1e-46
            
    def min_bl(self, mdm, sdm):
        fsol = fsolve(bl̂_func, x0=[0.001], args=(self.nobs, 
                                                 self.λsg(mdm, sdm),
                                                 self.ΔE, self.exp))
        return fsol[0]

    def min_sdm(self, mdm):
        minz = minimize(lsdm̂_func, x0=[-45], args=(self.nobs, 
                                                  self.λsg0(mdm),
                                                  self.ΔE, self.exp))
        return math.pow(10, minz.x[0])

    def min_sdmbl(self, mdm):
        sdm = self.min_sdm(mdm)
        bl = self.min_bl(mdm, sdm)
        return (sdm, bl)

    def min_mdmsdmbl(self):
        Sdm, Bl, Nll = [],[],[]
        for mdm in self.Mdm:
            sdm = self.min_sdm(mdm)
            bl = self.min_bl(mdm, sdm)
            nll = nlL1(bl, self.nobs, self.λsg(mdm, sdm), self.exp)
            Sdm.append(sdm)
            Bl.append(bl)
            Nll.append(nll)
        Nll = np.array(Nll)
        indx = np.where(Nll == np.min(Nll))[0]
        return (Mdm[indx], Sdm[indx], Bl[indx])  

In [5]:
nr_init = Nr('Xe', vE=mwd['vE'], 
                     vdfE=mwd['vdfE_50'],
                     vesc=p50(mwd['vescs']),
                     vcirc=p50(mwd['vcircs']),
                     rhosun=p50(mwd['rhosuns']),
                     Ethr=0.1, Eroi=5.0, ω=10.0)

### a. $\Delta m_{stat}$

In [6]:
stats = mk_mocks('stat', nr_init)


  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
from project.recoil import MLE

In [8]:
Mdm, Sdm = [], []
for mock in tqdm(stats[0]):
    mle = MLE(mock, nr_init, globalmin='precise', likelihood=1)

  0%|          | 0/1000 [00:00<?, ?it/s]

Running with chainlen = 508
Running with chainlen = 508
Running with chainlen = 508
Running with chainlen = 508


KeyboardInterrupt: 